In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
###USER DEFINED VARIABLES####
outdir = '/Users/stephaniecrilly/Kortemme_lab/helix_sliding/20240304_ngs_data'
#import dfs
ordered_seqs_df = pd.read_csv('/Users/stephaniecrilly/Kortemme_lab/helix_sliding/20240304_ngs_data/20240304_all_seqs_r1_order.csv')
myc_pos_df = pd.read_csv('/Users/stephaniecrilly/Kortemme_lab/helix_sliding/20240304_ngs_data/30-991152039/UniqueSeq/r1-hs-mycpos_Unique_AA.csv')
myc_neg_df = pd.read_csv('/Users/stephaniecrilly/Kortemme_lab/helix_sliding/20240304_ngs_data/30-991152039/UniqueSeq/r1-hs-mycneg_Unique_AA.csv')
ns_df  = pd.read_csv('/Users/stephaniecrilly/Kortemme_lab/helix_sliding/20240304_ngs_data/30-991152039/UniqueSeq/r1-hs-ns_Unique_AA.csv')

In [ ]:
#drop na
myc_pos_df = myc_pos_df.dropna()
myc_neg_df = myc_neg_df.dropna()
ns_df = ns_df.dropna()

#get only design sequences of interest
#search translated seqs for ones which start and end with flanking aa
mys_pos_df = myc_pos_df[myc_pos_df['Unique Amino Acid'].str.startswith('SASHM')]
mys_ned_df = myc_neg_df[myc_neg_df['Unique Amino Acid'].str.startswith('SASHM')]
ns_df = ns_df[ns_df['Unique Amino Acid'].str.startswith('SASHM')]

myc_pos_df = myc_pos_df[myc_pos_df['Unique Amino Acid'].str.endswith('LEGGG')]
myc_neg_df = myc_neg_df[myc_neg_df['Unique Amino Acid'].str.endswith('LEGGG')]
ns_df = ns_df[ns_df['Unique Amino Acid'].str.endswith('LEGGG')]

#append trailing amino acid to ordered seqs
ordered_seqs_df['sequence'] = 'SASHM' + ordered_seqs_df['aa_sequence'].astype(str) + 'LEGGG'

#get only seqs in library
myc_pos_df = pd.merge(ordered_seqs_df, myc_pos_df, left_on='sequence', right_on='Unique Amino Acid', how='inner')
myc_neg_df = pd.merge(ordered_seqs_df, myc_neg_df, left_on='sequence', right_on='Unique Amino Acid', how='inner')
ns_df = pd.merge(ordered_seqs_df, ns_df, left_on='sequence', right_on='Unique Amino Acid', how='inner')

myc_pos_df = myc_pos_df.drop('Unique Amino Acid', axis=1)
myc_neg_df = myc_neg_df.drop('Unique Amino Acid', axis=1)
ns_df = ns_df.drop('Unique Amino Acid', axis=1)

#add column for condition 
norm_condition = 'unsorted'
coi_1 = 'myc_pos'
coi_2 = 'myc_neg'

coi_1_df = myc_pos_df
coi_2_df = myc_neg_df
norm_condition_df = ns_df

norm_condition_df['Sort_condition'] = norm_condition
coi_1_df['Sort_condition'] = coi_1
coi_2_df['Sort_condition'] = coi_2

for col in norm_condition_df.columns:
    print(col)

In [ ]:
#norm all counts to total counts
norm_condition_df[f"norm_counts_{norm_condition}"] = norm_condition_df[' Unique Amino Acid Count'] /  norm_condition_df[' Unique Amino Acid Count'].sum()
coi_1_df[f"norm_counts_{coi_1}"] = coi_1_df[' Unique Amino Acid Count'] /  coi_1_df[' Unique Amino Acid Count'].sum()
coi_2_df[f"norm_counts_{coi_2}"] = coi_2_df[' Unique Amino Acid Count'] /  coi_2_df[' Unique Amino Acid Count'].sum()

norm_condition_df.to_csv(f'{outdir}/test_2.csv')
coi_1_df.to_csv(f'{outdir}/test_3.csv')

merge_df = pd.merge(norm_condition_df, coi_1_df, on=["Name", "sequence"])

merge_df[f"{coi_1}_norm"] = merge_df[f"norm_counts_{coi_1}"] /  merge_df[f"norm_counts_{norm_condition}"]
merge_df[f"{coi_1}_norm_log"] = np.log2(merge_df[f"{coi_1}_norm"])

merge_df[['Name', 'min_category']] = merge_df['Name'].str.split('min_', n=1, expand=True)
merge_df[['min_category', 'loop']] = merge_df['min_category'].str.split('_loop', n=1, expand=True)

merge_df = merge_df.sort_values(['Name']).reset_index()
merge_df.to_csv(f'{outdir}/test.csv')
print(merge_df.shape)

sns.barplot(merge_df, x='Name', y='myc_pos_norm_log', ci=None)
plt.show()
plt.clf()

sns.violinplot(merge_df, x='min_category', y='myc_pos_norm_log')
plt.show()
plt.clf()

sns.histplot(merge_df, x='myc_pos_norm_log', kde=True, bins=20)
plt.show()